*Copyright 2024 The Penzai Authors.*

*Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at*

> http://www.apache.org/licenses/LICENSE-2.0

*Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or  implied.
See the License for the specific language governing permissions and
limitations under the License.*

---

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-deepmind/penzai/blob/main/notebooks/jitting_and_sharding.ipynb) [![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/google-deepmind/penzai/blob/main/notebooks/jitting_and_sharding.ipynb)

# Jitting and Sharding Penzai Models

Penzai is designed to be compatible with JAX's standard function transformations, including JIT-compilation and array sharding. If you're already familiar with JIT compilation and distributed arrays in JAX, you shouldn't have to learn anything fundamentally new to apply it to Penzai! But Penzai does provide some utilities to make it easier to construct and manipulate shardings for Penzai models.

This notebook walks through some of the common aspects of JIT-compilation and sharding as they apply to Penzai tools and Penzai models. It assumes some basic familiarity with JAX's [JIT compilation](https://jax.readthedocs.io/en/latest/notebooks/thinking_in_jax.html) and [distributed array](https://jax.readthedocs.io/en/latest/notebooks/Distributed_arrays_and_automatic_parallelization.html) systems.

## Setup

Before we can get started in earnest, we need to set up the environment.

### Imports

To run this notebook, you need a Python environment with `penzai` and its dependencies installed.

In Colab or Kaggle, you can install it using the following command:

In [1]:
try:
  import penzai
except ImportError:
  !pip install penzai[notebook]

In [2]:
from __future__ import annotations

In [3]:
import dataclasses

import jax
import jax.numpy as jnp
import optax

In [4]:
import penzai
from penzai import pz

In [5]:
from penzai.example_models import gemma
from penzai.example_models import simple_mlp
from penzai.toolshed import basic_training

### Setting up Penzai

For this tutorial, we'll enable Treescope (Penzai's pretty-printer) as the default IPython pretty-printer. This is recommended when using Penzai in an interactive environment. We'll also enable automatic array visualization, which also makes it easy to visualize array shardings.

In [6]:
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.ts.register_context_manager_magic()

In [7]:
pz.enable_interactive_context()
pz.ts.active_autovisualizer.set_interactive(pz.ts.ArrayAutovisualizer())

We'll assume this notebook is running on a backend with eight devices. If needed, you can force JAX to treat the CPU backend as multiple devices using
```python
os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count=8"
```

In [8]:
pz.show(jax.local_devices())
assert jax.local_device_count() == 8

TreescopeRenderingFigure(child=PreventWordWrap(child=Siblings(children=(WithDynamicSharedPip(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root)', annotation='Copy path: '), IndentedChildren(children=[Siblings(children=(HyperlinkTarget(child=RoundtripCondition(roundtrip=WithBlockColor(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root)', annotation='Copy path: '), ColoredBorderIndentedChildren(children=[Siblings(children=(Text(text='original_repr'), Text(text='='), HyperlinkTarget(child=StringLiteralColor(child=Text(text="'TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)'")), keypath=(GetAttrKey(name='original_repr'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_repr)', annotation='Copy path: '),))))), Siblings(children=(Text(text='original_id'), Text(text='='), HyperlinkTarget(child=NumberColor(child=Text(text='140541543803264')), keypath=(GetAttrKey(name='original_id'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_id)', annotation='Copy path: '),))))), Siblings(children=(Text(text='original_type'), Text(text='='), HyperlinkTarget(child=FoldableTreeNodeImpl(contents=FoldCondition(collapsed=EmptyPart(), expanded=IndentedChildren(children=[Siblings(children=(HyperlinkTarget(child=AbbreviationColor(child=Text(text="<class 'jaxlib.xla_extension.Device'>")), keypath=(GetAttrKey(name='original_type'),)), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_type)', annotation='Copy path: '),)))))])), label=CommentColorWhenExpanded(child=Siblings(children=(FoldCondition(collapsed=EmptyPart(), expanded=Text(text='# ')), SummarizableCondition(summary=EmptyPart(), detail=QualifiedTypeNameSpanGroup(child=Text(text='jaxlib.xla_extension.'))), Text(text='Device')))), expand_state=<ExpandState.COLLAPSED: 'collapsed'>), keypath=(GetAttrKey(name='original_type'),)), FoldCondition(collapsed=EmptyPart(), expanded=Text(text=',')), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_type)', annotation='Copy path: '),)))))]), ColoredBottomLineSpanGroup(child=Text(text=')')))), label=HyperlinkTarget(child=ColoredTopLineSpanGroup(child=Siblings(children=(RoundtripCondition(roundtrip=QualifiedTypeNameSpanGroup(child=Text(text='penzai.treescope.copypaste_fallback.')), not_roundtrip=EmptyPart()), Text(text='NotRoundtrippable'), Text(text='(')))), keypath=()), expand_state=<ExpandState.WEAKLY_COLLAPSED: 'weakly_collapsed'>), color='#db845a'), not_roundtrip=AbbreviationColor(child=Text(text='TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)'))), keypath=(SequenceKey(idx=0),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root[0])', annotation='Copy path: '), CommentColor(child=Text(text='  # <jaxlib.xla_extension.Device object at 0x7fd260cb6980>'))))))), Siblings(children=(HyperlinkTarget(child=RoundtripCondition(roundtrip=WithBlockColor(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root)', annotation='Copy path: '), ColoredBorderIndentedChildren(children=[Siblings(children=(Text(text='original_repr'), Text(text='='), HyperlinkTarget(child=StringLiteralColor(child=Text(text="'TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1)'")), keypath=(GetAttrKey(name='original_repr'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCond

## JIT-Compiling Penzai Models

By convention, Penzai models are always JAX PyTrees with only arraylike leaves. This means you can always JIT-compile a function that takes a Penzai model as input or returns one as output, using ordinary `jax.jit`.

For instance, suppose we have the following Penzai model definition:

In [9]:
mlp_def = simple_mlp.MLP.from_config([8, 32, 32, 8])
mlp_def

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[
        Linear(weights=UninitializedParameter(initializer=functools.partial(<function variance_scaling_initializer at 0x7fd321ac3eb0>, scale=1.0, mode='fan_avg', distribution='uniform', input_axes={'features': 8}, output_axes={'features_out': 32}, parallel_axes={}, convolution_spatial_axes={}, dtype=<class 'jax.numpy.float32'>), name='Affine_0.Linear.weights', value_structure=<ArraySpec float32(| features:8, features_out:32)>), in_axis_names=('features',), out_axis_names=('features_out',)),
        RenameAxes(old=('features_out',), new=('features',)),
        AddBias(bias=UninitializedParameter(initializer=<function AddBias.from_config.<locals>.bias_initializer at 0x7fd321bce3b0>, name='Affine_0.AddBias.bias', value_structure=<ArraySpec float32(| features:32)>), new_axis_names=()),
      ],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=UninitializedParameter(initializer=functools.partial(<function variance_scaling_initializer at 0x7fd321ac3eb0>, scale=1.0, mode='fan_avg', distribution='uniform', input_axes={'features': 32}, output_axes={'features_out': 32}, parallel_axes={}, convolution_spatial_axes={}, dtype=<class 'jax.numpy.float32'>), name='Affine_1.Linear.weights', value_structure=<ArraySpec float32(| features:32, features_out:32)>), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=UninitializedParameter(initializer=<function AddBias.from_config.<locals>.bias_initializer at 0x7fc94c15caf0>, name='Affine_1.AddBias.bias', value_structure=<ArraySpec float32(| features:32)>), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=UninitializedParameter(initializer=functools.partial(<function variance_scaling_initializer at 0x7fd321ac3eb0>, scale=1.0, mode='fan_avg', distribution='uniform', input_axes={'features': 32}, output_axes={'features_out': 8}, parallel_axes={}, convolution_spatial_axes={}, dtype=<class 'jax.numpy.float32'>), name='Affine_2.Linear.weights', value_structure=<ArraySpec float32(| features:32, features_out:8)>), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=UninitializedParameter(initializer=<function AddBias.from_config.<locals>.bias_initializer at 0x7fc94c15dbd0>, name='Affine_2.AddBias.bias', value_structure=<ArraySpec float32(| features:8)>), new_axis_names=())],
    ),
  ],
)

We could JIT-compile the initializer for it:

In [10]:
@jax.jit
def init_my_model(mlp_def):
  return pz.nn.initialize_parameters(mlp_def, jax.random.key(0))

In [11]:
mlp = init_my_model(mlp_def)
mlp

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[
        Linear(weights=Parameter(value=<NamedArray float32(| features:8, features_out:32) ≈-0.0019 ±0.22 [≥-0.38, ≤0.38] nonzero:256 (wrapping jax.Array)>, name='Affine_0.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)),
        RenameAxes(old=('features_out',), new=('features',)),
        AddBias(bias=Parameter(value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, name='Affine_0.AddBias.bias'), new_axis_names=()),
      ],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:32) ≈-0.0037 ±0.18 [≥-0.31, ≤0.31] nonzero:1_024 (wrapping jax.Array)>, name='Affine_1.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, name='Affine_1.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:8) ≈-0.0052 ±0.23 [≥-0.38, ≤0.39] nonzero:256 (wrapping jax.Array)>, name='Affine_2.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:8) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:8 (wrapping jax.Array)>, name='Affine_2.AddBias.bias'), new_axis_names=())],
    ),
  ],
)

And we can just as easily JIT-compile a loss function that uses it:

In [12]:
# Just for demonstration; a real loss function would probably be more complex
# and involve a batch of examples

@jax.jit
def simple_mse_loss(mlp, inputs, target):
  output = mlp(inputs)
  diffs = (output - target).untag("features").unwrap()
  return jnp.sum(jnp.square(diffs))

In [13]:
simple_mse_loss(mlp, pz.nx.ones({"features": 8}), pz.nx.zeros({"features": 8}))

<jax.Array(1.0930231, dtype=float32)>

Note that Penzai models store their parameters inside the model, not in a separate parameter dictionary. This means you probably don't want to do this:
```python
# !!!! PROBABLY NOT WHAT YOU WANT TO DO:
jitted_call = jax.jit(mlp)
jitted_call(some_input)
```
The reason is that this will "bake in" the parameters of your MLP as constants in the compiled function, so JAX will need to recompile it if you update the parameters of the MLP.

Instead, you can do something like this:

In [14]:
@jax.jit
def jitted_call(mlp, arg):
  return mlp(arg)

In [15]:
jitted_call(mlp, pz.nx.ones({"features": 8}))

<NamedArray float32(| features:8) ≈0.1 ±0.36 [≥-0.43, ≤0.71] nonzero:8 (wrapping jax.Array)
  <Arrayviz rendering>
>

To save you the trouble of doing this manually when you want to JIT your model's `__call__`, Penzai provides a wrapper that does this automatically:

In [16]:
from penzai.toolshed import jit_wrapper

In [17]:
jitted_mlp = jit_wrapper.Jitted(mlp)
jitted_mlp(pz.nx.ones({"features": 8}))

<NamedArray float32(| features:8) ≈0.1 ±0.36 [≥-0.43, ≤0.71] nonzero:8 (wrapping jax.Array)
  <Arrayviz rendering>
>

`Jitted` is actually just an ordinary Penzai layer. It holds your model inside it as an attribute, and jit-compiles `__call__`:

```python
@pz.pytree_dataclass
class Jitted(pz.Layer):
  body: pz.LayerLike

  def __call__(self, argument: Any, /) -> Any:
    return jitted_call(self.body, argument)

```
You can see the model stored inside it as well:

In [18]:
jitted_mlp

Jitted(
  body=MLP( # Sequential
    sublayers=[
      Affine( # Sequential
        sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:8, features_out:32) ≈-0.0019 ±0.22 [≥-0.38, ≤0.38] nonzero:256 (wrapping jax.Array)>, name='Affine_0.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, name='Affine_0.AddBias.bias'), new_axis_names=())],
      ),
      Elementwise(fn=jax.nn.relu),
      Affine( # Sequential
        sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:32) ≈-0.0037 ±0.18 [≥-0.31, ≤0.31] nonzero:1_024 (wrapping jax.Array)>, name='Affine_1.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, name='Affine_1.AddBias.bias'), new_axis_names=())],
      ),
      Elementwise(fn=jax.nn.relu),
      Affine( # Sequential
        sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:8) ≈-0.0052 ±0.23 [≥-0.38, ≤0.39] nonzero:256 (wrapping jax.Array)>, name='Affine_2.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:8) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:8 (wrapping jax.Array)>, name='Affine_2.AddBias.bias'), new_axis_names=())],
      ),
    ],
  ),
)

It defines `__call__` to be JIT-compiled, including itself as a non-static argument. This means that JAX will automatically re-used the cached compiled program if you call multiple `Jitted` layers with the same structure, even if you update the parameters.

It will also re-compile if you make modifications. For instance, we can freely insert new logic into our "jitted MLP", and those new functions will run under JIT as well:

In [19]:
@pz.pytree_dataclass
class PrintMyValue(pz.Layer):
  def __call__(self, arg):
    pz.show("Intermediate:", arg)
    return arg

In [20]:
patched_jitted_mlp = (
    pz.select(jitted_mlp)
    .at_instances_of(pz.nn.Elementwise)
    .insert_before(PrintMyValue())
)

In [21]:
patched_jitted_mlp

Jitted(
  body=MLP( # Sequential
    sublayers=[
      Affine( # Sequential
        sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:8, features_out:32) ≈-0.0019 ±0.22 [≥-0.38, ≤0.38] nonzero:256 (wrapping jax.Array)>, name='Affine_0.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, name='Affine_0.AddBias.bias'), new_axis_names=())],
      ),
      PrintMyValue(),
      Elementwise(fn=jax.nn.relu),
      Affine( # Sequential
        sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:32) ≈-0.0037 ±0.18 [≥-0.31, ≤0.31] nonzero:1_024 (wrapping jax.Array)>, name='Affine_1.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, name='Affine_1.AddBias.bias'), new_axis_names=())],
      ),
      PrintMyValue(),
      Elementwise(fn=jax.nn.relu),
      Affine( # Sequential
        sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:8) ≈-0.0052 ±0.23 [≥-0.38, ≤0.39] nonzero:256 (wrapping jax.Array)>, name='Affine_2.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:8) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:8 (wrapping jax.Array)>, name='Affine_2.AddBias.bias'), new_axis_names=())],
      ),
    ],
  ),
)

In [22]:
patched_jitted_mlp(pz.nx.ones({"features": 8}))

TreescopeRenderingFigure(child=PreventWordWrap(child=Siblings(children=(Text(text='Intermediate:'), Text(text=' '), HyperlinkTarget(child=FoldableTreeNodeImpl(contents=SummarizableCondition(summary=EmptyPart(), detail=Siblings(children=(IndentedChildren(children=[Siblings(children=(Text(text='named_axes'), Text(text='='), WithDynamicSharedPip(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.named_axes)', annotation='Copy path: '), IndentedChildren(children=[Siblings(children=(HyperlinkTarget(child=StringLiteralColor(child=Text(text="'features'")), keypath=None), Text(text=': '), HyperlinkTarget(child=NumberColor(child=Text(text='32')), keypath=(GetAttrKey(name='named_axes'), DictKey(key='features'))), FoldCondition(collapsed=EmptyPart(), expanded=Text(text=',')), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string="(lambda root: root.named_axes['features'])", annotation='Copy path: '),)))))]), Text(text='})'))), label=HyperlinkTarget(child=Siblings(children=(RoundtripCondition(roundtrip=QualifiedTypeNameSpanGroup(child=Text(text='collections.')), not_roundtrip=EmptyPart()), Text(text='OrderedDict'), Text(text='({'))), keypath=(GetAttrKey(name='named_axes'),)), expand_state=<ExpandState.COLLAPSED: 'collapsed'>), node_id=140497780861632, seen_more_than_once=set()), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(DynamicSharedCheck(if_shared=SharedWarningLabel(child=Text(text=' # Repeated python obj at 0x7fc83051eac0')), node_id=140497780861632, seen_more_than_once=set()), StringCopyButton(copy_string='(lambda root: root.named_axes)', annotation='Copy path: ')))))), Siblings(children=(PyTreeNodeFieldName(child=Text(text='data_array')), Text(text='='), HyperlinkTarget(child=RoundtripCondition(roundtrip=WithBlockColor(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root)', annotation='Copy path: '), ColoredBorderIndentedChildren(children=[Siblings(children=(Text(text='original_repr'), Text(text='='), HyperlinkTarget(child=StringLiteralColor(child=Text(text="'Traced<ShapedArray(float32[32])>with<DynamicJaxprTrace(level=1/0)>'")), keypath=(GetAttrKey(name='original_repr'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_repr)', annotation='Copy path: '),))))), Siblings(children=(Text(text='original_id'), Text(text='='), HyperlinkTarget(child=NumberColor(child=Text(text='140496434932176')), keypath=(GetAttrKey(name='original_id'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_id)', annotation='Copy path: '),))))), Siblings(children=(Text(text='original_type'), Text(text='='), HyperlinkTarget(child=FoldableTreeNodeImpl(contents=FoldCondition(collapsed=EmptyPart(), expanded=IndentedChildren(children=[Siblings(children=(HyperlinkTarget(child=AbbreviationColor(child=Text(text="<class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>")), keypath=(GetAttrKey(name='original_type'),)), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_type)', annotation='Copy path: '), CommentColor(child=Siblings(children=(Text(text='  # Defined at '), Text(text='line 1663 of /usr/local/lib/python3.10/dist-packages/jax/_src/interpreters/partial_eval.py')))))))))])), label=CommentColorWhenExpanded(child=Siblings(children=(FoldCondition(collapsed=EmptyPart(), expanded=Text(text='# ')), SummarizableCondition(summary=EmptyPart(), detail=QualifiedTypeNameSpanGroup(child=Text(text='jax.interpreters.partial_eval.'))), Text(text='DynamicJa

TreescopeRenderingFigure(child=PreventWordWrap(child=Siblings(children=(Text(text='Intermediate:'), Text(text=' '), HyperlinkTarget(child=FoldableTreeNodeImpl(contents=SummarizableCondition(summary=EmptyPart(), detail=Siblings(children=(IndentedChildren(children=[Siblings(children=(Text(text='named_axes'), Text(text='='), WithDynamicSharedPip(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.named_axes)', annotation='Copy path: '), IndentedChildren(children=[Siblings(children=(HyperlinkTarget(child=StringLiteralColor(child=Text(text="'features'")), keypath=None), Text(text=': '), HyperlinkTarget(child=NumberColor(child=Text(text='32')), keypath=(GetAttrKey(name='named_axes'), DictKey(key='features'))), FoldCondition(collapsed=EmptyPart(), expanded=Text(text=',')), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string="(lambda root: root.named_axes['features'])", annotation='Copy path: '),)))))]), Text(text='})'))), label=HyperlinkTarget(child=Siblings(children=(RoundtripCondition(roundtrip=QualifiedTypeNameSpanGroup(child=Text(text='collections.')), not_roundtrip=EmptyPart()), Text(text='OrderedDict'), Text(text='({'))), keypath=(GetAttrKey(name='named_axes'),)), expand_state=<ExpandState.COLLAPSED: 'collapsed'>), node_id=140497783310656, seen_more_than_once=set()), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(DynamicSharedCheck(if_shared=SharedWarningLabel(child=Text(text=' # Repeated python obj at 0x7fc830774940')), node_id=140497783310656, seen_more_than_once=set()), StringCopyButton(copy_string='(lambda root: root.named_axes)', annotation='Copy path: ')))))), Siblings(children=(PyTreeNodeFieldName(child=Text(text='data_array')), Text(text='='), HyperlinkTarget(child=RoundtripCondition(roundtrip=WithBlockColor(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root)', annotation='Copy path: '), ColoredBorderIndentedChildren(children=[Siblings(children=(Text(text='original_repr'), Text(text='='), HyperlinkTarget(child=StringLiteralColor(child=Text(text="'Traced<ShapedArray(float32[32])>with<DynamicJaxprTrace(level=1/0)>'")), keypath=(GetAttrKey(name='original_repr'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_repr)', annotation='Copy path: '),))))), Siblings(children=(Text(text='original_id'), Text(text='='), HyperlinkTarget(child=NumberColor(child=Text(text='140496568724880')), keypath=(GetAttrKey(name='original_id'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_id)', annotation='Copy path: '),))))), Siblings(children=(Text(text='original_type'), Text(text='='), HyperlinkTarget(child=FoldableTreeNodeImpl(contents=FoldCondition(collapsed=EmptyPart(), expanded=IndentedChildren(children=[Siblings(children=(HyperlinkTarget(child=AbbreviationColor(child=Text(text="<class 'jax._src.interpreters.partial_eval.DynamicJaxprTracer'>")), keypath=(GetAttrKey(name='original_type'),)), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_type)', annotation='Copy path: '), CommentColor(child=Siblings(children=(Text(text='  # Defined at '), Text(text='line 1663 of /usr/local/lib/python3.10/dist-packages/jax/_src/interpreters/partial_eval.py')))))))))])), label=CommentColorWhenExpanded(child=Siblings(children=(FoldCondition(collapsed=EmptyPart(), expanded=Text(text='# ')), SummarizableCondition(summary=EmptyPart(), detail=QualifiedTypeNameSpanGroup(child=Text(text='jax.interpreters.partial_eval.'))), Text(text='DynamicJa

<NamedArray float32(| features:8) ≈0.1 ±0.36 [≥-0.43, ≤0.71] nonzero:8 (wrapping jax.Array)
  <Arrayviz rendering>
>

And you can always pull the model back out of the `Jitted` wrapper:

In [23]:
jitted_mlp.body

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[
        Linear(weights=Parameter(value=<NamedArray float32(| features:8, features_out:32) ≈-0.0019 ±0.22 [≥-0.38, ≤0.38] nonzero:256 (wrapping jax.Array)>, name='Affine_0.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)),
        RenameAxes(old=('features_out',), new=('features',)),
        AddBias(bias=Parameter(value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, name='Affine_0.AddBias.bias'), new_axis_names=()),
      ],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:32) ≈-0.0037 ±0.18 [≥-0.31, ≤0.31] nonzero:1_024 (wrapping jax.Array)>, name='Affine_1.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:32) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:32 (wrapping jax.Array)>, name='Affine_1.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:8) ≈-0.0052 ±0.23 [≥-0.38, ≤0.39] nonzero:256 (wrapping jax.Array)>, name='Affine_2.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:8) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:8 (wrapping jax.Array)>, name='Affine_2.AddBias.bias'), new_axis_names=())],
    ),
  ],
)

In [24]:
assert jitted_mlp.body is mlp

## Sharding Basics, and Visualizing Shardings with Treescope

Penzai's array autovisualizer supports showing shardings and sharded arrays by default. This section explains the basics of JAX's distributed array shardings and how you can visualize the different components in Treescope. (See [this page](https://jax.readthedocs.io/en/latest/notebooks/Distributed_arrays_and_automatic_parallelization.html) for the official documentation of JAX's sharding system.)

### Positional shardings

At a high level, you can think of a "sharding" as a multidimensional array of device objects, which will be matched with your multidimensional array of data to determine which part of the array ends up on each device. You generally build a sharding by starting with a NumPy array of devices:

In [25]:
from jax.experimental import mesh_utils
devices = mesh_utils.create_device_mesh((8,))
devices

array([TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0),
       TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1),
       TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0),
       TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1),
       TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0),
       TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1),
       TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0),
       TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1)],
      dtype=object)

A simple type of sharding is `PositionalSharding`, which essentially just holds onto these devices and tracks some extra JAX-specific information. If you print out a `PositionalSharding` in Treescope, it color-codes the devices and shows you their arrangement:

In [26]:
pos_sharding = jax.sharding.PositionalSharding(devices)
pos_sharding

PositionalSharding([{TPU 0} {TPU 1} {TPU 2} {TPU 3} {TPU 6} {TPU 7} {TPU 4} {TPU 5}], shape=(8,))

In this case, the sharding has a single positional axis, of length 8. We can use this to shard arrays whose (first) positional axis is a multiple of 8. For instance:

In [27]:
jax.device_put(jnp.ones(16), pos_sharding)

<Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],      dtype=float32)
  <Arrayviz rendering>
  Sharded across 8 TPU devices (click to expand)
>

You can click the "Sharded across 8 TPU devices" message to show a visualization of the sharding for this array. When automatic array visualization is enabled, sharding visualizations are automatically added to any array that is sharded or replicated.

We can reshape positional shardings to give them multiple axes:

In [28]:
pos_sharding.reshape((4,2))

PositionalSharding([[{TPU 0} {TPU 1}]
                    [{TPU 2} {TPU 3}]
                    [{TPU 6} {TPU 7}]
                    [{TPU 4} {TPU 5}]], shape=(4, 2))

In [29]:
jax.device_put(jnp.ones([8, 8]), pos_sharding.reshape((4,2)))

<jax.Array float32(8, 8) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64
  <Arrayviz rendering>
  Sharded across 8 TPU devices (click to expand)
>

If you expand the sharding visualization above, you'll see how the two axes of the array are matched with the two axes of the sharding.

You can also use shardings to indicate that certain parts of the array should be *replicated* on multiple devices, using `replicate`:

In [30]:
pos_sharding.reshape((2, 4)).replicate(axis=0)

PositionalSharding([[{TPU 0, 6} {TPU 1, 7} {TPU 2, 4} {TPU 3, 5}]], shape=(1, 4))

In [31]:
jax.device_put(jnp.ones([8, 8]), pos_sharding.reshape((2, 4)).replicate(axis=0))

<jax.Array float32(8, 8) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64
  <Arrayviz rendering>
  Sharded across 8 TPU devices (click to expand)
>

Each element of an array with a replicated sharding will appear on more than one device. This is visually represented in Treescope using a multicolored pattern.

You can also fully-replicate the array over all of the devices:

In [32]:
pos_sharding.replicate(axis=0)

PositionalSharding([{TPU 0, 1, 2, 3, 4, 5, 6, 7}], shape=(1,))

In [33]:
jax.device_put(jnp.ones([8, 8]), pos_sharding.replicate(axis=0))

<jax.Array float32(8, 8) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64
  <Arrayviz rendering>
  Replicated across 8 TPU devices (click to expand)
>

Fully-replicated arrays are also identified as such in the sharding summary before being expanded.

### Meshes and named shardings

It is often convenient to refer to different axes of an array of devices by name instead of by position. JAX represents this using the type `jax.sharding.Mesh`. Conceptually, just as a `PositionalSharding` is essentially a positional array of devices, a `Mesh` is essentially a named array of devices, i.e. an array of devices where each axis has a name.

Penzai annotates the device ID arrays of `Mesh` instances with axis names instead of axis positions:

In [34]:
mesh = jax.sharding.Mesh(devices.reshape((4, 2)), axis_names=('foo', 'bar'))
mesh

Mesh(device_ids=array([[0, 1],
       [2, 3],
       [6, 7],
       [4, 5]]), axis_names=('foo', 'bar'))

To shard a (positionally-indexed) JAX array using a mesh, you can use `jax.sharding.NamedSharding` to assign particular axis indices to mesh axis names, like this:

In [35]:
jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec('foo', 'bar'))

NamedSharding(mesh=Mesh('foo': 4, 'bar': 2), spec=PartitionSpec('foo', 'bar'))

In [36]:
jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec(None, ('bar', 'foo'), None))

NamedSharding(mesh=Mesh('foo': 4, 'bar': 2), spec=PartitionSpec(None, ('bar', 'foo'), None))

In [37]:
jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec('foo'))

NamedSharding(mesh=Mesh('foo': 4, 'bar': 2), spec=PartitionSpec('foo',))

Note: Each `NamedSharding` specifies how to shard an input array's *positional axes*, since ordinary JAX arrays only have positional axes. The names in the `NamedSharding` are just a way to match the positional axes in the array with the corresponding names in the `Mesh`. For this reason, visualizations of `NamedSharding` instances are annotated with positional axes, not axis names.

(Penzai already has its own mechanism for binding names to an array's positional axes: `pz.nx.NamedArray`. We'll discuss how to shard Penzai's `NamedArray` next.)

## Sharding Penzai's NamedArrays

### Manually sharding NamedArrays

Fundamentally, there are no changes when applying JAX shardings to Penzai's `NamedArray`s. Internally, a `NamedArray` is just a dataclass PyTree node that contains a JAX array and some axis name annotations, which we can see if we disable automatic array visualization temporarily:

In [38]:
arr = pz.nx.arange("foo", 1, 4) + pz.nx.arange("bar", 0, 4)

In [39]:
# With automatic array visualization enabled:
arr

<NamedArray int32(| foo:3, bar:4) [≥1, ≤6] nonzero:12 (wrapping jax.Array)
  <Arrayviz rendering>
>

In [40]:
%%autovisualize None
# ^ With automatic array visualization disabled (and expanding it to show detail)
pz.select(arr).at_instances_of(jax.Array).show_value()

JAX's sharding system allows you to specify the sharding for a PyTree of arrays by using a matching PyTree of shardings. So, we can build a sharding for this named array by inserting a positional sharding into it:

In [41]:
data_array_sharding = jax.sharding.PositionalSharding(devices).reshape((2,4)).replicate(axis=0)
sharding_for_arr = pz.nx.NamedArray(
    named_axes=arr.named_axes,
    data_array=data_array_sharding,
)
sharding_for_arr

NamedArray(
  named_axes=OrderedDict({'foo': 3, 'bar': 4}),
  data_array=<PositionalSharding([[{TPU 0, 6} {TPU 1, 7} {TPU 2, 4} {TPU 3, 5}]], shape=(1, 4))
    <Arrayviz rendering>
  >,
)

Applying this sharding to the NamedArray shards the `data_array` attribute (try expanding below):

In [42]:
%%autovisualize lambda a,p: pz.ts.ArrayAutovisualizer()(a, p) if isinstance(a, jax.Array) else None
# (^ this line overrides the autovisualizer to show the sharding of the data array when expanded)

sharded_arr = jax.device_put(arr, sharding_for_arr)
pz.select(sharded_arr).at_instances_of(jax.Array).show_value()

But with normal automatic array visualization, treescope will show you how the *named* axes are sharded, since that's usually what you care about when using Penzai models in practice:

In [43]:
sharded_arr

<NamedArray int32(| foo:3, bar:4) [≥1, ≤6] nonzero:12 (wrapping jax.Array)
  <Arrayviz rendering>
  Sharded across 8 TPU devices (click to expand)
>

### Automatically building shardings for NamedArrays

To simplify this process, Penzai provides some optional utilities for constructing shardings for `NamedArray` instances. These utilities take a `Mesh`, and allow you to map from `NamedArray` axis names to `Mesh` axis names across a tree of arrays.

For instance, consider this tree of arrays:

In [44]:
some_array_tree = {
    "one": pz.nx.ones({"a": 4, "b": 8, "c": 6}),
    "two": pz.nx.ones({"a": 8}),
    "three": pz.nx.ones({"b": 4, "d": 12}),
}
some_array_tree

{'one': <NamedArray float32(| a:4, b:8, c:6) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:192 (wrapping jax.Array)
   <Arrayviz rendering>
 >,
 'two': <NamedArray float32(| a:8) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:8 (wrapping jax.Array)
   <Arrayviz rendering>
 >,
 'three': <NamedArray float32(| b:4, d:12) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:48 (wrapping jax.Array)
   <Arrayviz rendering>
 >}

And this mesh:

In [45]:
mesh = jax.sharding.Mesh(devices.reshape((4, 2)), axis_names=('foo', 'bar'))
mesh

Mesh(device_ids=array([[0, 1],
       [2, 3],
       [6, 7],
       [4, 5]]), axis_names=('foo', 'bar'))

We can assign each named axis in `some_array_tree` to an axis in the mesh using the `name_to_name_sharding` utility, which builds a tree of shardings that is compatible with the tree of arrays:

In [46]:
from penzai.toolshed import sharding_util

In [47]:
shardings = sharding_util.name_to_name_sharding(
    some_array_tree,
    mesh,
    axis_name_to_mesh_name={
        "a": "bar",
        "b": "foo",
    },
)
shardings

{'one': NamedArray(
   named_axes=OrderedDict({'a': 4, 'b': 8, 'c': 6}),
   data_array=<NamedSharding(mesh=Mesh('foo': 4, 'bar': 2), spec=PartitionSpec('bar', 'foo', None))
     <Arrayviz rendering>
   >,
 ),
 'three': NamedArray(
   named_axes=OrderedDict({'b': 4, 'd': 12}),
   data_array=<NamedSharding(mesh=Mesh('foo': 4, 'bar': 2), spec=PartitionSpec('foo', None))
     <Arrayviz rendering>
   >,
 ),
 'two': NamedArray(
   named_axes=OrderedDict({'a': 8}),
   data_array=<NamedSharding(mesh=Mesh('foo': 4, 'bar': 2), spec=PartitionSpec('bar',))
     <Arrayviz rendering>
   >,
 )}

We can then apply those shardings to the original array tree to shard the corresponding axes:

In [48]:
jax.device_put(some_array_tree, shardings)

{'one': <NamedArray float32(| a:4, b:8, c:6) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:192 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'three': <NamedArray float32(| b:4, d:12) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:48 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'two': <NamedArray float32(| a:8) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:8 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >}

Even simpler, if you just want to call `device_put` you can bundle them into one call:

In [49]:
sharding_util.name_to_name_device_put(
    some_array_tree,
    mesh,
    axis_name_to_mesh_name={
        "a": "bar",
        "b": "foo",
    },
)

{'one': <NamedArray float32(| a:4, b:8, c:6) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:192 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'three': <NamedArray float32(| b:4, d:12) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:48 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'two': <NamedArray float32(| a:8) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:8 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >}

If your mesh happens to use the exact same axis names as your arrays, you don't need the `axis_name_to_mesh_name` argument:

In [50]:
already_matching_mesh = jax.sharding.Mesh(devices.reshape((4, 2)), axis_names=('b', 'a'))
sharding_util.name_to_name_device_put(
    some_array_tree,
    already_matching_mesh,
    # axis_name_to_mesh_name inferred as {"a":"a", "b":"b"}
)

{'one': <NamedArray float32(| a:4, b:8, c:6) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:192 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'three': <NamedArray float32(| b:4, d:12) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:48 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'two': <NamedArray float32(| a:8) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:8 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >}

## Sharding Penzai Models and Training Loops

Penzai also provides some utilities that are specific to training and using Penzai neural newtork models. These are simple self-contained utilities that can be a good starting point, but you are free to customize them to get lower-level control when needed.

### Sharding Parameter Initializers

Uninitialized Penzai models directly expose all of the parameter initializers to you as attributes inside your model. If you want to customize the sharding of your parameters, you can JIT-compile the initializer with the appropriate sharding, e.g.
```python
sharded_initializer = jax.jit(
  pz.nn.initialize_parameters,
  out_shardings=..., # <- insert your desired sharding specification here
)
params = sharded_initializer(mlp_def, jax.random.key(42))
```
If you want to infer `out_shardings` using the axis names of your parameters, you can do that using the helper function `initialize_parameters_sharded`. This function just traces the initializer to figure out the parameter shapes, infers the right sharding to use, and then runs your initializer accordingly.

For instance, here's how you could initialize the parameters of a small transformer in a sharded way:

In [51]:
# Using the Gemma model architecture, but very small for demonstration purposes.
tiny_transformer_def = gemma.model_core.GemmaTransformer.from_config(
    gemma.model_core.GemmaTransformerConfig(
        num_heads=2,
        embedding_dim=64,
        projection_dim=16,
        single_kv_head=False,
        mlp_hidden_dim=128,
        num_decoder_blocks=2,
        vocab_size=100,
        parameter_dtype=jnp.float32,
        activation_dtype=jnp.float32,
    )
)

In [52]:
tiny_transformer = sharding_util.initialize_parameters_sharded(
    tiny_transformer_def,
    jax.random.key(42),
    mesh=jax.sharding.Mesh(devices, axis_names=('devices',)),
    axis_name_to_mesh_name={
        # Shard the embedding dimension across devices.
        "embedding": "devices",
    },
)
tiny_transformer

GemmaTransformer(
  #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
  # Input: GemmaInputs(tokens=<tokens>, positions=<positions>, attention_mask=<attention mask>)
  # Output: <unnormalized logits>
  #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
  config=GemmaTransformerConfig(
    num_heads=2,
    embedding_dim=64,
    projection_dim=16,
    single_kv_head=False,
    mlp_hidden_dim=128,
    num_decoder_blocks=2,
    vocab_size=100,
    parameter_dtype=jax.numpy.float32,
    activation_dtype=jax.numpy.float32,
  ),
  body=WithSideInputsFromInputTuple(handler_id='WithSideInputsFromInputTuple_a6b36', body=WithConstantSideInputs(handler_id='shared_params_f7a6b', body=Sequential(sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=SharedParameterLookup(ref=HandledSideInputRef(handler_id='shared_params_f7a6b', tag=SharedParamTag(name='embedder.embeddings')), value_structure=<ArraySpec float32(| vocabulary:100, embedding:64)>), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(8., dtype=float32)>), GemmaTransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_attention_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), GemmaAttention(input_to_query=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.query.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions')), ConstantRescale(by=<jax.Array(0.25, dtype=float32)>)]), input_to_key=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.key.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions'))]), input_to_value=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.value.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection'))]), query_key_to_attn=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'projection': 'p'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}), ApplyAttentionMask(mask=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='attn_mask'), masked_out_value=<jax.Array(-2.3819763e+38, dtype=float32)>), Softmax(axes='kv_seq')]), attn_value_to_output=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'projection': 'p'}), Linear(weights=Parameter(value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.output.weights'), in_axis_names=('heads', 'projection'), out_axis_names=('embedding',))]))])), Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_ffw_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), GemmaFeed

### Sharding Training Steps

You're encouraged to write your own custom training loop for your use case. However, the basic training step implementation in `penzai.toolshed.basic_training` does support sharded training.

The easiest way to shard a training loop is to just shard your model parameters and inputs, JIT-compile the training loop, and let JAX figure out how the sharding should propagate. XLA can usually automatically infer a decent sharding for the computation and its outputs.

For instance, here's how we could write a simple training loop for this tiny transformer:

In [53]:
# Simple loss function for demonstration purposes.
def simplified_xent_loss_fn(model, rng, state, input_examples):
  del rng, state  # Unused.
  # Run the model.
  outputs = model(gemma.model_core.GemmaInputs.from_basic_segments(
      input_examples[{"seq": pz.slice[:-1]}]
  ))
  # Compute log-probabilities along the "vocabulary" axis.
  all_log_probs = pz.nx.nmap(jax.nn.log_softmax)(
      outputs.untag("vocabulary")
  ).tag("vocabulary")
  # Index by the correct tokens.
  correct_next_tokens = input_examples[{"seq": pz.slice[1:]}]
  correct_log_probs = all_log_probs[{"vocabulary": correct_next_tokens}]
  # Take averages.
  loss = -correct_log_probs.untag("batch", "seq").unwrap().mean()
  return loss, None, {"loss": loss}

In [54]:
train_step = basic_training.build_train_step_fn(
    simplified_xent_loss_fn,
    jit=True,
    # donate_params_and_state=True,  # <- Uncomment to allow XLA memory optimizations.
)
train_state = basic_training.TrainState.initial_state(
    model=tiny_transformer,
    optimizer_def=optax.adamw(5e-5, weight_decay=0.01),
    root_rng=jax.random.key(42),
)

In [55]:
# Take a training step (with a dummy input in this case).
input_examples = pz.nx.ones({"batch": 8, "seq": 20}, dtype=jnp.int32)
updated_train_state, outs = train_step(train_state, input_examples=input_examples)

In [56]:
# Show the updated parameters.
pz.select(updated_train_state.model).at_instances_of(pz.nn.Parameter).get_sequence()

(Parameter(
   value=<NamedArray float32(| embedding:64) ≈1.0 ±5e-05 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.pre_attention_norm.scale.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.attn.query.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.attn.key.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.attn.value.weights',
 ),
 Parameter(
   value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.attn.output.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64) ≈1.0 ±5e-05 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.pre_ffw_norm.scale.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, neurons:128) ≈-0.0015 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.mlp.gating_linear.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, neurons:128) ≈0.0002 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.mlp.value_linear.weights',
 ),
 Parameter(
   value=<NamedArray float32(| neurons:128, embedding:64) ≈-0.00032 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.mlp.out_linear.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64) ≈1.0 ±4.9e-05 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.pre_attention_norm.scale.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈0.0023 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.attn.query.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈0.0031 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.attn.key.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.00019 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.attn.value.weights',
 ),
 Parameter(
   value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈-0.0033 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.attn.output.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64) ≈1.0 ±4.8e-05 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.pre_ffw_norm.scale.weights',
 ),
 Parameter

If you inspect the parameters above, you will likely see that they are still sharded along the embedding axis (because XLA will likely infer that that keeping the same sharding is the easiest).

If you prefer, however, you can also manually specify what shardings you want to use, and the training step function will respect them. For instance, we can explicitly request that the model parameters and optimizer states be sharded across the "features" axis, and the inputs be sharded across the "batch" axis.

In [57]:
mesh = jax.sharding.Mesh(devices, axis_names=('devices',))
train_step = basic_training.build_train_step_fn(
    simplified_xent_loss_fn,
    jit=True,
    # Shard inputs over "batch" axis.
    input_kwarg_shardings={
        "input_examples": sharding_util.name_to_name_sharding(
            input_examples,
            mesh,
            axis_name_to_mesh_name={"batch": "devices"},
        ),
    },
    # Shard model and optimizer params over "embedding" axis.
    train_state_shardings=sharding_util.name_to_name_sharding(
        train_state,
        mesh,
        axis_name_to_mesh_name={"embedding": "devices"},
        ignore_unnamed_arrays=True,
    ),
    # donate_params_and_state=True,  # <- Uncomment to allow XLA memory optimizations.
)

In [58]:
# Take a training step (with a dummy input in this case).
input_examples = pz.nx.ones({"batch": 8, "seq": 20}, dtype=jnp.int32)
updated_train_state, outs = train_step(train_state, input_examples=input_examples)
# Show the updated parameters.
pz.select(updated_train_state.model).at_instances_of(pz.nn.Parameter).get_sequence()

(Parameter(
   value=<NamedArray float32(| embedding:64) ≈1.0 ±5e-05 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.pre_attention_norm.scale.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.attn.query.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.attn.key.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.attn.value.weights',
 ),
 Parameter(
   value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.attn.output.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64) ≈1.0 ±5e-05 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.pre_ffw_norm.scale.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, neurons:128) ≈-0.0015 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.mlp.gating_linear.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, neurons:128) ≈0.0002 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.mlp.value_linear.weights',
 ),
 Parameter(
   value=<NamedArray float32(| neurons:128, embedding:64) ≈-0.00032 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_0.mlp.out_linear.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64) ≈1.0 ±4.9e-05 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.pre_attention_norm.scale.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈0.0023 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.attn.query.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈0.0031 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.attn.key.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.00019 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.attn.value.weights',
 ),
 Parameter(
   value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈-0.0033 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.attn.output.weights',
 ),
 Parameter(
   value=<NamedArray float32(| embedding:64) ≈1.0 ±4.8e-05 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
     <Arrayviz rendering>
     Sharded across 8 TPU devices (click to expand)
   >,
   name='block_1.pre_ffw_norm.scale.weights',
 ),
 Parameter

### Adding Sharding Constraints to Models

You may want more control over the way that intermediate values are sharded. JAX allows you to control this using `jax.lax.with_sharding_constraint`, which forces a particular value to have a particular sharding.

In a Penzai model, sharding constraints can be enforced by simply inserting new layers into the model at the points where you want to constrain the shardings. Penzai's `sharding_util` module provides two simple classes `ConstrainSharding` and `ConstrainShardingByName` for this purpose, defined as
```python
@pz.pytree_dataclass
class ConstrainSharding(pz.Layer):
  sharding: PyTreeOfShardings = field(metadata={"pytree_node": False})
  def __call__(self, tree: Any) -> Any:
    return jax.lax.with_sharding_constraint(tree, self.sharding)

@pz.pytree_dataclass
class ConstrainShardingByName(pz.Layer):
  mesh: jax.sharding.Mesh = field(metadata={"pytree_node": False})
  axis_name_to_mesh_name: dict[str, str | tuple[str, ...]] | None = (
      field(default=None, metadata={"pytree_node": False})
  )
  def __call__(self, tree: PyTreeOfNamedArrays) -> PyTreeOfNamedArrays:
    return jax.lax.with_sharding_constraint(
        tree,
        name_to_name_sharding(tree, self.mesh, self.axis_name_to_mesh_name),
    )
```

You can insert them into the model using logic like this:

In [59]:
# Make sure it's sharded over the batch axis after each block.
tiny_transformer_constrained = (
    pz.select(tiny_transformer)
    .at_instances_of(gemma.model_core.GemmaTransformerBlock)
    .insert_after(sharding_util.ConstrainShardingByName(
        mesh, axis_name_to_mesh_name={"batch": "devices"}
    ))
)

In [60]:
# Visualize the constraints:
pz.select(tiny_transformer_constrained).at_instances_of(sharding_util.ConstrainShardingByName)

Selection(
  selected_by_path=OrderedDict({
    (
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='body'),
      GetAttrKey(name='sublayers'),
      SequenceKey(idx=3),
    ):
      ConstrainShardingByName(mesh=<Mesh(device_ids=array([0, 1, 2, 3, 6, 7, 4, 5]), axis_names=('devices',))>, axis_name_to_mesh_name={'batch': 'devices'}),
    (GetAttrKey(name='body'), GetAttrKey(name='body'), GetAttrKey(name='body'), GetAttrKey(name='sublayers'), SequenceKey(idx=5)):
      ConstrainShardingByName(mesh=<Mesh(device_ids=array([0, 1, 2, 3, 6, 7, 4, 5]), axis_names=('devices',))>, axis_name_to_mesh_name={'batch': 'devices'}),
  }),
  remainder=GemmaTransformer(config=GemmaTransformerConfig(num_heads=2, embedding_dim=64, projection_dim=16, single_kv_head=False, mlp_hidden_dim=128, num_decoder_blocks=2, vocab_size=100, parameter_dtype=jax.numpy.float32, activation_dtype=jax.numpy.float32), body=WithSideInputsFromInputTuple(handler_id='WithSideInputsFromInputTuple_a6b36', body=WithConstantSideInputs(handler_id='shared_params_f7a6b', body=Sequential(sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=SharedParameterLookup(ref=HandledSideInputRef(handler_id='shared_params_f7a6b', tag=SharedParamTag(name='embedder.embeddings')), value_structure=<ArraySpec float32(| vocabulary:100, embedding:64)>), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(8., dtype=float32)>), GemmaTransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_attention_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), GemmaAttention(input_to_query=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.query.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions')), ConstantRescale(by=<jax.Array(0.25, dtype=float32)>)]), input_to_key=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.key.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions'))]), input_to_value=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.value.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection'))]), query_key_to_attn=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'projection': 'p'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}), ApplyAttentionMask(mask=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='attn_mask'), masked_out_value=<jax.Array(-2.3819763e+38, dtype=float32)>), Softmax(axes='kv_seq')]), attn_value_to_output=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'projection': 'p'}), Linear(weights=Parameter(value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.output.weights'), in_axis_names=('heads', 'proj

This gives you a version of the model whose intermediates will be sharded in the way you specified.

In [61]:
train_state = basic_training.TrainState.initial_state(
    model=tiny_transformer_constrained,
    optimizer_def=optax.adamw(5e-5, weight_decay=0.01),
    root_rng=jax.random.key(42),
)

In [62]:
mesh = jax.sharding.Mesh(devices, axis_names=('devices',))
train_step = basic_training.build_train_step_fn(
    simplified_xent_loss_fn,
    jit=True,
    # Shard inputs over "batch" axis.
    input_kwarg_shardings={
        "input_examples": sharding_util.name_to_name_sharding(
            input_examples,
            mesh,
            axis_name_to_mesh_name={"batch": "devices"},
        ),
    },
    # Shard model and optimizer params over "embedding" axis.
    train_state_shardings=sharding_util.name_to_name_sharding(
        train_state,
        mesh,
        axis_name_to_mesh_name={"embedding": "devices"},
        ignore_unnamed_arrays=True,
    ),
    # donate_params_and_state=True,  # <- Uncomment to allow XLA memory optimizations.
)

In [63]:
# Take a training step (with a dummy input in this case).
input_examples = pz.nx.ones({"batch": 8, "seq": 20}, dtype=jnp.int32)
updated_train_state, outs = train_step(train_state, input_examples=input_examples)

If you later want to change how your model's intermediates are sharded, you can simply remove these constraints:

In [64]:
tiny_transformer_unconstrained = (
    pz.select(tiny_transformer_constrained)
    .at_instances_of(sharding_util.ConstrainShardingByName)
    .remove_from_parent()
)

# No more constraints:
(
    pz.select(tiny_transformer_unconstrained)
    .at_instances_of(sharding_util.ConstrainShardingByName)
    .assert_count_is(0)
)

Selection(
  selected_by_path=OrderedDict({}),
  remainder=GemmaTransformer(
    #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
    # Input: GemmaInputs(tokens=<tokens>, positions=<positions>, attention_mask=<attention mask>)
    # Output: <unnormalized logits>
    #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
    config=GemmaTransformerConfig(num_heads=2, embedding_dim=64, projection_dim=16, single_kv_head=False, mlp_hidden_dim=128, num_decoder_blocks=2, vocab_size=100, parameter_dtype=jax.numpy.float32, activation_dtype=jax.numpy.float32),
    body=WithSideInputsFromInputTuple(handler_id='WithSideInputsFromInputTuple_a6b36', body=WithConstantSideInputs(handler_id='shared_params_f7a6b', body=Sequential(sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=SharedParameterLookup(ref=HandledSideInputRef(handler_id='shared_params_f7a6b', tag=SharedParamTag(name='embedder.embeddings')), value_structure=<ArraySpec float32(| vocabulary:100, embedding:64)>), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(8., dtype=float32)>), GemmaTransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_attention_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), GemmaAttention(input_to_query=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.query.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions')), ConstantRescale(by=<jax.Array(0.25, dtype=float32)>)]), input_to_key=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.key.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions'))]), input_to_value=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.value.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection'))]), query_key_to_attn=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'projection': 'p'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}), ApplyAttentionMask(mask=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='attn_mask'), masked_out_value=<jax.Array(-2.3819763e+38, dtype=float32)>), Softmax(axes='kv_seq')]), attn_value_to_output=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'projection': 'p'}), Linear(weights=Parameter(value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.output.weights'), in_axis_names=('heads', 'projection'), out_axis_names=('embedding',))]))])), Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_ffw_norm.scale.weights'), in_axis_names=(), o

### Aside: Parameter checkpointing for sharded models

Note: To make it easier to save and restore parameters from checkpoints even if you've inserted sharding constraints (or made other modifications), we recommend only checkpointing the dictionary of model parameters, not the full model structure. This is how the `basic_training.TrainState` stores the parameters internally:

In [65]:
train_state

TrainState(
  step=0,
  root_rng=# jax.Array key<fry>()
    Array((), dtype=key<fry>) overlaying:
    [ 0 42]
  ,
  params={'block_0.pre_attention_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, 'block_0.attn.query.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, 'block_0.attn.key.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, 'block_0.attn.value.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, 'block_0.attn.output.weights': <NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, 'block_0.pre_ffw_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, 'block_0.mlp.gating_linear.weights': <NamedArray float32(| embedding:64, neurons:128) ≈-0.0015 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>, 'block_0.mlp.value_linear.weights': <NamedArray float32(| embedding:64, neurons:128) ≈0.0002 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>, 'block_0.mlp.out_linear.weights': <NamedArray float32(| neurons:128, embedding:64) ≈-0.00032 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>, 'block_1.pre_attention_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, 'block_1.attn.query.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈0.0023 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, 'block_1.attn.key.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈0.0031 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, 'block_1.attn.value.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.00019 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, 'block_1.attn.output.weights': <NamedArray float32(| heads:2, projection:16, embedding:64) ≈-0.0033 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, 'block_1.pre_ffw_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, 'block_1.mlp.gating_linear.weights': <NamedArray float32(| embedding:64, neurons:128) ≈-5.8e-05 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>, 'block_1.mlp.value_linear.weights': <NamedArray float32(| embedding:64, neurons:128) ≈-0.00077 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>, 'block_1.mlp.out_linear.weights': <NamedArray float32(| neurons:128, embedding:64) ≈0.00047 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)>, 'final_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, 'embedder.embeddings': <NamedArray float32(| vocabulary:100, embedding:64) ≈0.0005 ±0.13 [≥-0.43, ≤0.46] nonzero:6_400 (wrapping jax.Array)>},
  model_without_params=GemmaTransformer(config=GemmaTransformerConfig(num_heads=2, embedding_dim=64, projection_dim=16, single_kv_head=False, mlp_hidden_dim=128, num_decoder_blocks=2, vocab_size=100, parameter_dtype=jax.numpy.float32, activation_dtype=jax.numpy.float32), body=WithSideInputsFromInputTuple(handler_id='WithSideInputsFromInputTuple_a6b36', body=WithConstantSideInputs(handler_id='shared_params_f7a6b', body=Sequential(sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=SharedParameterLookup(ref=HandledSideInputRef(handler_id='shared_params_f7a6b', tag=SharedParamTag(name='embedder.embeddings')), value_structure=<ArraySpec float32(| vocabulary:100, embedding:64)>), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(8., dtype=float32)>), GemmaTransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding

You can manually extract the parameter dictionary from a model like this:

In [66]:
param_dict = {
    param.name: param.value
    for param in (
        pz.select(tiny_transformer)
        .at_instances_of(pz.nn.Parameter)
        .get_sequence()
    )
}
param_dict

{'block_0.pre_attention_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_0.attn.query.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_0.attn.key.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_0.attn.value.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_0.attn.output.weights': <NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_0.pre_ffw_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_0.mlp.gating_linear.weights': <NamedArray float32(| embedding:64, neurons:128) ≈-0.0015 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_0.mlp.value_linear.weights': <NamedArray float32(| embedding:64, neurons:128) ≈0.0002 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_0.mlp.out_linear.weights': <NamedArray float32(| neurons:128, embedding:64) ≈-0.00032 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.pre_attention_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.attn.query.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈0.0023 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.attn.key.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈0.0031 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.attn.value.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.00019 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.attn.output.weights': <NamedArray float32(| heads:2, projection:16, embedding:64) ≈-0.0033 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.pre_ffw_norm.scale.weights': <NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.mlp.gating_linear.weights': <NamedArray float32(| embedding:64, neurons:128) ≈-5.8e-05 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.mlp.value_linear.weights': <NamedArray float32(| embedding:64, neurons:128) ≈-0.00077 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wrapping jax.Array)
   <Arrayviz rendering>
   Sharded across 8 TPU devices (click to expand)
 >,
 'block_1.mlp.out_linear.weights': <NamedArray float32(| neurons:128, embedding:64) ≈0.00047 ±0.1 [≥-0.18, ≤0.18] nonzero:8_192 (wr

And later restore them using something like this:

In [67]:
restored = (
    pz.select(tiny_transformer)
    .at_instances_of(pz.nn.Parameter)
    .apply(
        lambda param: dataclasses.replace(param, value=param_dict[param.name])
    )
)
restored

GemmaTransformer(
  #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
  # Input: GemmaInputs(tokens=<tokens>, positions=<positions>, attention_mask=<attention mask>)
  # Output: <unnormalized logits>
  #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
  config=GemmaTransformerConfig(
    num_heads=2,
    embedding_dim=64,
    projection_dim=16,
    single_kv_head=False,
    mlp_hidden_dim=128,
    num_decoder_blocks=2,
    vocab_size=100,
    parameter_dtype=jax.numpy.float32,
    activation_dtype=jax.numpy.float32,
  ),
  body=WithSideInputsFromInputTuple(handler_id='WithSideInputsFromInputTuple_a6b36', body=WithConstantSideInputs(handler_id='shared_params_f7a6b', body=Sequential(sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=SharedParameterLookup(ref=HandledSideInputRef(handler_id='shared_params_f7a6b', tag=SharedParamTag(name='embedder.embeddings')), value_structure=<ArraySpec float32(| vocabulary:100, embedding:64)>), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(8., dtype=float32)>), GemmaTransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_attention_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), GemmaAttention(input_to_query=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.query.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions')), ConstantRescale(by=<jax.Array(0.25, dtype=float32)>)]), input_to_key=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.key.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions'))]), input_to_value=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.value.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection'))]), query_key_to_attn=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'projection': 'p'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}), ApplyAttentionMask(mask=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='attn_mask'), masked_out_value=<jax.Array(-2.3819763e+38, dtype=float32)>), Softmax(axes='kv_seq')]), attn_value_to_output=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'projection': 'p'}), Linear(weights=Parameter(value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.output.weights'), in_axis_names=('heads', 'projection'), out_axis_names=('embedding',))]))])), Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_ffw_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), GemmaFeed

If you haven't yet ininitialized your parameters, you can do something similar to initialize the `UninitializedParameter`s directly using saved values:

In [68]:
(
    pz.select(tiny_transformer_def)
    .at_instances_of(pz.nn.UninitializedParameter)
    .apply(
        lambda uninit: uninit.initialize_with_value(param_dict[uninit.name])
    )
)

GemmaTransformer(
  #╭┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╮
  # Input: GemmaInputs(tokens=<tokens>, positions=<positions>, attention_mask=<attention mask>)
  # Output: <unnormalized logits>
  #╰┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄┄╯
  config=GemmaTransformerConfig(
    num_heads=2,
    embedding_dim=64,
    projection_dim=16,
    single_kv_head=False,
    mlp_hidden_dim=128,
    num_decoder_blocks=2,
    vocab_size=100,
    parameter_dtype=jax.numpy.float32,
    activation_dtype=jax.numpy.float32,
  ),
  body=WithSideInputsFromInputTuple(handler_id='WithSideInputsFromInputTuple_a6b36', body=WithConstantSideInputs(handler_id='shared_params_f7a6b', body=Sequential(sublayers=[EmbeddingLookup(table=EmbeddingTable(embeddings=SharedParameterLookup(ref=HandledSideInputRef(handler_id='shared_params_f7a6b', tag=SharedParamTag(name='embedder.embeddings')), value_structure=<ArraySpec float32(| vocabulary:100, embedding:64)>), vocabulary_axis='vocabulary')), ConstantRescale(by=<jax.Array(8., dtype=float32)>), GemmaTransformerBlock(sublayers=[Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_attention_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), GemmaAttention(input_to_query=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-1.4e-05 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.query.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions')), ConstantRescale(by=<jax.Array(0.25, dtype=float32)>)]), input_to_key=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0082 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.key.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection')), ApplyRoPE(embedding_axis='projection', max_wavelength=10000, positions=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='token_positions'))]), input_to_value=Sequential(sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| embedding:64, heads:2, projection:16) ≈-0.0057 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.value.weights'), in_axis_names=('embedding',), out_axis_names=('heads', 'projection'))]), query_key_to_attn=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'projection': 'p'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}), ApplyAttentionMask(mask=HandledSideInputRef(handler_id='WithSideInputsFromInputTuple_a6b36', tag='attn_mask'), masked_out_value=<jax.Array(-2.3819763e+38, dtype=float32)>), Softmax(axes='kv_seq')]), attn_value_to_output=Sequential(sublayers=[NamedEinsum(input_axes=({'seq': 'tq', 'heads': 'h', 'kv_seq': 'tkv'}, {'seq': 'tkv', 'heads': 'h', 'projection': 'p'}), output_axes={'seq': 'tq', 'heads': 'h', 'projection': 'p'}), Linear(weights=Parameter(value=<NamedArray float32(| heads:2, projection:16, embedding:64) ≈0.00078 ±0.14 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='block_0.attn.output.weights'), in_axis_names=('heads', 'projection'), out_axis_names=('embedding',))]))])), Residual(delta=Sequential(sublayers=[RMSLayerNorm(sublayers=[RMSStandardize(across=('embedding',), epsilon=<jax.Array(1.e-06, dtype=float32)>), Linear(weights=Parameter(value=<NamedArray float32(| embedding:64) ≈1.0 ±0.0 [≥1.0, ≤1.0] nonzero:64 (wrapping jax.Array)>, name='block_0.pre_ffw_norm.scale.weights'), in_axis_names=(), out_axis_names=())]), GemmaFeed

You can also use this to build a PyTree with the same shape as your model parameter dictionary without initializing them first:

In [69]:
# Produces a structure containing jax.ShapeDtypeStruct
param_dict_structure = {
    uninit.name: uninit.as_empty_parameter().value
    for uninit in (
        pz.select(tiny_transformer_def)
        .at_instances_of(pz.nn.UninitializedParameter)
        .get_sequence()
    )
}
param_dict_structure

{'block_0.pre_attention_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_0.attn.query.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_0.attn.key.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_0.attn.value.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_0.attn.output.weights': <NamedArray float32(| heads:2, projection:16, embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_0.pre_ffw_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_0.mlp.gating_linear.weights': <NamedArray float32(| embedding:64, neurons:128) (wrapping ShapeDtypeStruct)>,
 'block_0.mlp.value_linear.weights': <NamedArray float32(| embedding:64, neurons:128) (wrapping ShapeDtypeStruct)>,
 'block_0.mlp.out_linear.weights': <NamedArray float32(| neurons:128, embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_1.pre_attention_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_1.attn.query.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_1.attn.key.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_1.attn.value.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_1.attn.output.weights': <NamedArray float32(| heads:2, projection:16, embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_1.pre_ffw_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_1.mlp.gating_linear.weights': <NamedArray float32(| embedding:64, neurons:128) (wrapping ShapeDtypeStruct)>,
 'block_1.mlp.value_linear.weights': <NamedArray float32(| embedding:64, neurons:128) (wrapping ShapeDtypeStruct)>,
 'block_1.mlp.out_linear.weights': <NamedArray float32(| neurons:128, embedding:64) (wrapping ShapeDtypeStruct)>,
 'final_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'embedder.embeddings': <NamedArray float32(| vocabulary:100, embedding:64) (wrapping ShapeDtypeStruct)>}

This, in turn, could be used to build a sharding specification:

In [70]:
param_dict_sharding = sharding_util.name_to_name_sharding(
    param_dict_structure,
    mesh,
    axis_name_to_mesh_name={"embedding": "devices"},
)
param_dict_sharding

{'block_0.attn.key.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'heads': 2, 'projection': 16}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None, None))
     <Arrayviz rendering>
   >,
 ),
 'block_0.attn.output.weights': NamedArray(
   named_axes=OrderedDict({'heads': 2, 'projection': 16, 'embedding': 64}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec(None, None, 'devices'))
     <Arrayviz rendering>
   >,
 ),
 'block_0.attn.query.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'heads': 2, 'projection': 16}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None, None))
     <Arrayviz rendering>
   >,
 ),
 'block_0.attn.value.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'heads': 2, 'projection': 16}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None, None))
     <Arrayviz rendering>
   >,
 ),
 'block_0.mlp.gating_linear.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'neurons': 128}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None))
     <Arrayviz rendering>
   >,
 ),
 'block_0.mlp.out_linear.weights': NamedArray(
   named_axes=OrderedDict({'neurons': 128, 'embedding': 64}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec(None, 'devices'))
     <Arrayviz rendering>
   >,
 ),
 'block_0.mlp.value_linear.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'neurons': 128}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None))
     <Arrayviz rendering>
   >,
 ),
 'block_0.pre_attention_norm.scale.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices',))
     <Arrayviz rendering>
   >,
 ),
 'block_0.pre_ffw_norm.scale.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices',))
     <Arrayviz rendering>
   >,
 ),
 'block_1.attn.key.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'heads': 2, 'projection': 16}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None, None))
     <Arrayviz rendering>
   >,
 ),
 'block_1.attn.output.weights': NamedArray(
   named_axes=OrderedDict({'heads': 2, 'projection': 16, 'embedding': 64}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec(None, None, 'devices'))
     <Arrayviz rendering>
   >,
 ),
 'block_1.attn.query.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'heads': 2, 'projection': 16}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None, None))
     <Arrayviz rendering>
   >,
 ),
 'block_1.attn.value.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'heads': 2, 'projection': 16}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None, None))
     <Arrayviz rendering>
   >,
 ),
 'block_1.mlp.gating_linear.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'neurons': 128}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None))
     <Arrayviz rendering>
   >,
 ),
 'block_1.mlp.out_linear.weights': NamedArray(
   named_axes=OrderedDict({'neurons': 128, 'embedding': 64}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec(None, 'devices'))
     <Arrayviz rendering>
   >,
 ),
 'block_1.mlp.value_linear.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64, 'neurons': 128}),
   data_array=<NamedSharding(mesh=Mesh('devices': 8), spec=PartitionSpec('devices', None))
     <Arrayviz rendering>
   >,
 ),
 'block_1.pre_attention_norm.scale.weights': NamedArray(
   named_axes=OrderedDict({'embedding': 64}),
   data_array

You can also wrap the `NamedSharding` leaves in a `jax.ShapeDtypeStruct`:

In [71]:
param_dict_sharding_structs = sharding_util.name_to_name_sharding(
    param_dict_structure,
    mesh,
    axis_name_to_mesh_name={"embedding": "devices"},
    as_shape_dtype_struct=True,  # <- Wraps shardings in ShapeDtypeStruct
)
param_dict_sharding_structs

{'block_0.attn.key.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_0.attn.output.weights': <NamedArray float32(| heads:2, projection:16, embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_0.attn.query.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_0.attn.value.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_0.mlp.gating_linear.weights': <NamedArray float32(| embedding:64, neurons:128) (wrapping ShapeDtypeStruct)>,
 'block_0.mlp.out_linear.weights': <NamedArray float32(| neurons:128, embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_0.mlp.value_linear.weights': <NamedArray float32(| embedding:64, neurons:128) (wrapping ShapeDtypeStruct)>,
 'block_0.pre_attention_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_0.pre_ffw_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_1.attn.key.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_1.attn.output.weights': <NamedArray float32(| heads:2, projection:16, embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_1.attn.query.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_1.attn.value.weights': <NamedArray float32(| embedding:64, heads:2, projection:16) (wrapping ShapeDtypeStruct)>,
 'block_1.mlp.gating_linear.weights': <NamedArray float32(| embedding:64, neurons:128) (wrapping ShapeDtypeStruct)>,
 'block_1.mlp.out_linear.weights': <NamedArray float32(| neurons:128, embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_1.mlp.value_linear.weights': <NamedArray float32(| embedding:64, neurons:128) (wrapping ShapeDtypeStruct)>,
 'block_1.pre_attention_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'block_1.pre_ffw_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>,
 'embedder.embeddings': <NamedArray float32(| vocabulary:100, embedding:64) (wrapping ShapeDtypeStruct)>,
 'final_norm.scale.weights': <NamedArray float32(| embedding:64) (wrapping ShapeDtypeStruct)>}

If you're using `orbax.checkpoint` for your parameters, you can configure it so that it restores the parameters directly using this sharding, using something like
```python
loaded_param_dict = checkpointer.restore(
    ckpt_path,
    args=orbax.checkpoint.args.StandardRestore(param_dict_sharding_structs),
)
```